In [1]:
import pandas as pd
import numpy as np

In [21]:
def compute_gini_index(list_mem):
    total = sum (list_mem)
    length = len(list_mem)
    gini = 0
    
    for i in range (0 , length):
        val = list_mem[i]
        if val == 0:
            continue
        p_num = val/total
        p_num_sq = np.square(p_num)
        gini += p_num_sq 
        
    gini = 1 - gini    
        
    return  gini 

In [22]:
def compute_target_gini_index (df, target_name):
    class_count = df[target_name].value_counts()
    unq = df[target_name].unique()
    
    dist = []
    for i in range (0, len(unq)):
       dist.append (class_count[unq[i]])
    print (dist)    
    gini = compute_gini_index(dist)
    return gini

In [23]:
def get_sub_class_df (df, feature_name):
    sub_features = df[feature_name].unique()
    #print (sub_features)
    
    sub_df = {}
    for i in range (0 , len(sub_features)):
        #print (sub_features[i])
        k = df[feature_name] == sub_features[i]
        #print (df[k])
        sub_df[sub_features[i]] = df[k]
    
    return sub_df , sub_features  

In [24]:
def compute_feature_gini_index (df, feature_name, target_name):
    
    #df1 = df.set_index (feature_name)
    #print (df1)
    
    df1 = df
    
    subcalss_df , sub_features = get_sub_class_df (df1, feature_name)
    #print (subcalss_df)
    target_class = df1[target_name].unique()
    
    
    total_gini = 0
    for i in range (0, len(sub_features)):
        df2 = subcalss_df[sub_features[i]]
        dist=[]
        #print (sub_features[i])
        for j in range (0, len(target_class)):
            #print (target_class[j])
            df3 = df2[df2[target_name] == target_class[j]]
            #print (df3)
            dist.append (df3.shape[0])
            
            
        #print (dist) 
        entropy = compute_gini_index(dist)
        #print (entropy)
        val = df2.shape[0]/df1[target_name].shape[0]
        #print (val)
        entropy *= val
        
        #print (df2.shape[0])
        #print (df[target_name].shape[0])
        
        total_gini += entropy
        
    return (total_gini)

In [25]:
#def compute_total_gini (entropy_parent, entropy):
#    return entropy_parent - entropy

In [47]:
def compute_split_feature(df):  
    class_name = None
    
    gini = 100
    best_gini = 100
    tot_gini  = 0
    split_feature = None
    for i in range (0, len(df.columns) - 1):
        gini = compute_feature_gini_index (df , df.columns[i], 'Play')    
    
        print ("Feature -- {} ::: Gini {}".format(df.columns[i], gini))
    
        if (gini == 0):
            best_gini = gini
            split_feature = df.columns[i]
        else :
            if (gini < best_gini):
                best_gini = gini                
                split_feature = df.columns[i]
        tot_gini += gini        
                

    if (tot_gini == 0.0):
        class_name = df.Play.unique()
        split_feature = None
        
        
    print ("Best Gini {} split column :: {}". format(best_gini, split_feature))
    return split_feature ,class_name

In [48]:

dict = {'Outlook':['Rainy','Rainy','Overcast','Sunny','Sunny','Sunny','Overcast','Rainy','Rainy','Sunny','Rainy','Overcast','Overcast','Sunny'],
       
        'Temp':['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild' ],
        
        'Humidity': ['High','High','High','High','Normal','Normal','Normal','High','Normal','Normal','Normal','High','Normal','High'],
        
        'Windy' : ['False' , 'True', 'False','False','False','True','True','False','False','False','True','True','False','True'],
       
        'Play' : ['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No', ]}


In [49]:
df = pd.DataFrame.from_dict (dict)

In [50]:
df

,Outlook,Temp,Humidity,Windy,Play
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


In [51]:
target_col = 'Play'
target_entropy = compute_target_gini_index (df , target_col)
print (target_entropy)

[5, 9]
0.4591836734693877


In [52]:
def compute_branch_split (df , target_col):
    print (df)
    target_entropy = compute_target_gini_index (df , target_col)
    print (target_entropy)
    
    root_node, class_name = compute_split_feature(df)
    
    if root_node == None:
        print (class_name)
        return class_name
    
    print ("************************************")
    print ("Root Node :: {}".format(root_node))
    print ("************************************")

    # split the df with respect to the root node 
    unq = df [root_node].unique()
    for i in range (0, len(unq)):
        df_sub = df[df[root_node] == unq[i]]
        print (unq[i])    
        df_sub.drop (root_node , axis = 1, inplace = True)
        next_node = compute_branch_split (df_sub , target_col)
        print ("************************************")
        print ("Next Node ::{} ---> {} ---> {}".format(root_node, unq[i], next_node))
        print ("************************************")
    
    return root_node

In [53]:
target_col = 'Play'

df_sub  = compute_branch_split(df, target_col)



     Outlook  Temp Humidity  Windy Play
0      Rainy   Hot     High  False   No
1      Rainy   Hot     High   True   No
2   Overcast   Hot     High  False  Yes
3      Sunny  Mild     High  False  Yes
4      Sunny  Cool   Normal  False  Yes
5      Sunny  Cool   Normal   True   No
6   Overcast  Cool   Normal   True  Yes
7      Rainy  Mild     High  False   No
8      Rainy  Cool   Normal  False  Yes
9      Sunny  Mild   Normal  False  Yes
10     Rainy  Mild   Normal   True  Yes
11  Overcast  Mild     High   True  Yes
12  Overcast   Hot   Normal  False  Yes
13     Sunny  Mild     High   True   No
[5, 9]
0.4591836734693877
Feature -- Outlook ::: Gini 0.34285714285714286
Feature -- Temp ::: Gini 0.44047619047619047
Feature -- Humidity ::: Gini 0.3673469387755103
Feature -- Windy ::: Gini 0.42857142857142855
Best Gini 0.34285714285714286 split column :: Outlook
************************************
Root Node :: Outlook
************************************
Rainy
    Temp Humidity  Windy Play
0 

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
